#**NAMA: FATHURRAHMAN NUR AZIZ**
#**NIM 20.11.3694**

##1. Buatlah spark session dengan nama "Responsi"

In [1]:
#install findspark and pyspark
!pip install -q findspark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.1 MB/s eta 0:00:00


In [94]:
#create spark session
from pyspark.sql import SparkSession
spark = SparkSession \
  .builder \
  .appName("Responsi") \
  .getOrCreate()

In [95]:
#create spark context
sc = spark.sparkContext

In [96]:
#import dataset
url = 'https://raw.githubusercontent.com/ariflaksito/ST153-Big-Data-Lanjut/master/flights_small.csv'
dir = r'/content/drive/MyDrive/Colab\ Notebooks/BDPA\ LANJUT/'

path = dir + url.split('/')[-1]

!curl {url} > {path}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  599k  100  599k    0     0  1966k      0 --:--:-- --:--:-- --:--:-- 1972k


In [97]:
#load dataset
df = spark.read.csv(path, header=True, inferSchema=True)

##2. Cek tipe data tiap kolom dan tampilkan 20 data teratas

In [98]:
#cek tipe data tiap kolom
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [99]:
#menampilkan 20 kolom teratas
df.show(20)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

##3. Buat RDD berisikan data Carrier, OriginAirportID, dan DestAirportID

In [100]:
#membuat RDD dari dataframe
RDD = df.rdd

#print tipe data RDD
print(f'The file type of RDD is {type(RDD)}')

The file type of RDD is <class 'pyspark.rdd.RDD'>


In [101]:
#lihat 5 data teratas RDD
RDD.take(5)

[Row(year=2014, month=12, day=8, dep_time='658', dep_delay='-7', arr_time='935', arr_delay='-5', carrier='VX', tailnum='N846VA', flight=1780, origin='SEA', dest='LAX', air_time='132', distance=954, hour='6', minute='58'),
 Row(year=2014, month=1, day=22, dep_time='1040', dep_delay='5', arr_time='1505', arr_delay='5', carrier='AS', tailnum='N559AS', flight=851, origin='SEA', dest='HNL', air_time='360', distance=2677, hour='10', minute='40'),
 Row(year=2014, month=3, day=9, dep_time='1443', dep_delay='-2', arr_time='1652', arr_delay='2', carrier='VX', tailnum='N847VA', flight=755, origin='SEA', dest='SFO', air_time='111', distance=679, hour='14', minute='43'),
 Row(year=2014, month=4, day=9, dep_time='1705', dep_delay='45', arr_time='1839', arr_delay='34', carrier='WN', tailnum='N360SW', flight=344, origin='PDX', dest='SJC', air_time='83', distance=569, hour='17', minute='5'),
 Row(year=2014, month=3, day=9, dep_time='754', dep_delay='-1', arr_time='1015', arr_delay='1', carrier='AS', ta

In [102]:
#membuat RDD_sub yang berisi hanya kolom carrier, origin, dan dest dari RDD sebelumnya
RDD_sub=RDD.map(lambda x: (x['carrier'],x['origin'],x['dest']))

#print tipe data RDD_sub
print(f'The file type of RDD_sub is {type(RDD_sub)}')

The file type of RDD_sub is <class 'pyspark.rdd.PipelinedRDD'>


In [103]:
#lihat 5 data teratas RDD_sub
RDD_sub.take(5)

[('VX', 'SEA', 'LAX'),
 ('AS', 'SEA', 'HNL'),
 ('VX', 'SEA', 'SFO'),
 ('WN', 'PDX', 'SJC'),
 ('AS', 'SEA', 'BUR')]

##4. Ubah RDD ke dataframe pada poin 3

In [104]:
#ubah RDD sub ke Dataframe dan tampung ke variabel df_sub
df_sub = RDD_sub.toDF(['Carrier','OriginAirportID','DestAirportID'])

#print tipe data df_sub
print(f'The file type of df_sub is {type(df_sub)}')

The file type of df_sub is <class 'pyspark.sql.dataframe.DataFrame'>


In [105]:
#cek tipe data dataframe df_sub
df_sub.printSchema()

root
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)



In [106]:
#lihat 5 row teratas dari df_sub
df_sub.show(5)

+-------+---------------+-------------+
|Carrier|OriginAirportID|DestAirportID|
+-------+---------------+-------------+
|     VX|            SEA|          LAX|
|     AS|            SEA|          HNL|
|     VX|            SEA|          SFO|
|     WN|            PDX|          SJC|
|     AS|            SEA|          BUR|
+-------+---------------+-------------+
only showing top 5 rows



##5. Tampilkan berapa jumlah masing-massing DestAirportID dari masing-masing Carrier!

In [120]:
#grouping by Carrier dan DestAirPortID, 
#kemudian menghitung jumlah dari DestAirportID
#kemudian menampilkan dengan sort ascending berdasarkan kolom carrier

count_dest_by_carrier = df_sub.groupBy('Carrier','DestAirportID')

count_dest_by_carrier \
  .count() \
  .sort('Carrier') \
  .show(100)

+-------+-------------+-----+
|Carrier|DestAirportID|count|
+-------+-------------+-----+
|     AA|          PHX|    1|
|     AA|          ORD|  156|
|     AA|          JFK|   17|
|     AA|          MIA|   19|
|     AA|          DFW|  289|
|     AS|          SLC|   59|
|     AS|          ANC|  377|
|     AS|          FLL|   29|
|     AS|          BLI|    5|
|     AS|          MSP|   35|
|     AS|          PHX|  163|
|     AS|          MSY|    9|
|     AS|          LIH|   37|
|     AS|          ORD|   86|
|     AS|          AUS|   27|
|     AS|          SAT|   18|
|     AS|          LAX|  346|
|     AS|          STL|   16|
|     AS|          BWI|   11|
|     AS|          DCA|   69|
|     AS|          OAK|   86|
|     AS|          OGG|   67|
|     AS|          DTW|    3|
|     AS|          JNU|   65|
|     AS|          GEG|   91|
|     AS|          LAS|  256|
|     AS|          MCO|   31|
|     AS|          KTN|   77|
|     AS|          DEN|   91|
|     AS|          TPA|   11|
|     AS| 